In [1]:
import numpy as np
import copy

In [83]:
class MultilayeredNeuralNetwork(object):
    
    activation_funs = {
        'tanh': np.tanh,
        'sigm': np.vectorize(lambda x: 1.0 / (1.0 + np.exp(-1.0 * x)))
    }
    
    def __init__(self, layers, activation_fun='tanh', alpha=0.15, beta=0.1):
        self.alpha = alpha
        self.beta = beta
        self.f = MultilayeredNeuralNetwork.activation_funs[activation_fun]
        self.f_prime = lambda x: 1 - self.f(x)
        self.num_layers = len(layers) - 1
        self.layers = []
        self.thetas = []
        self.xs = []
        self.ys = []
        self.deltas = []
        self.Deltas = []
        for l in range(self.num_layers):
            r = 2.4 / layers[l]
            size = (layers[l], layers[l + 1])
            w = np.random.random(size) - r
            self.thetas.append(np.random.random(layers[l + 1]) - r)
            self.layers.append(w)
            self.deltas.append(None)
            self.Deltas.append(None)
            
            
    def fit(self, X, Y, epochs=5000):
        for epoch in range(epochs):
            for x, actual_y in zip(X, Y):
                predicted_y = self.predict(x)
                error = actual_y - predicted_y
                self.deltas[-1] = self.ys[-1]
                self.deltas[-1] *= 1 - self.ys[-1]
                self.deltas[-1] *= error
                #self.Deltas[self.num_layers - 1] = self.alpha * self.xs[self.num_layers - 1].T.dot(self.deltas[self.num_layers - 1])
                #self.Deltas[-1] = self.alpha * np.reshape(self.xs[-1].T, (3, 1)) @ self.deltas[-1]
                #self.Deltas[-1] = np.reshape(self.Deltas[-1], (3, 1))
                #print(self.Deltas[-1], self.layers[-1], np.reshape(self.xs[-1].T, (3, 1)))
                for l in range(self.num_layers - 2, -1, -1):
                    self.deltas[l] = self.ys[l] * (1 - self.ys[l])
                    self.deltas[l] *= self.deltas[l + 1].dot(self.layers[l + 1].T)
                
                for l in range(self.num_layers):
                    rows, cols = self.layers[l].shape
                    x = np.reshape(self.xs[l].T, (rows, 1))
                    d = np.reshape(self.deltas[l], (1, cols))
                    self.Deltas[l] = self.alpha * x.dot(d)
                    self.layers[l] += self.Deltas[l]
                    self.thetas[l] += self.alpha * -1 * self.deltas[l]
                
        #return self.deltas, self.Deltas
                
    
    def predict(self, x):
        y = x 
        self.xs = []
        self.ys = []
        for layer, theta in zip(self.layers, self.thetas):
            self.xs.append(y)
            temp = np.dot(y, layer)
            temp -= theta
            y = self.f(temp)
            self.ys.append(y)
        return y
            
            
        
    

In [84]:
nn = MultilayeredNeuralNetwork([2, 2, 1])
X = np.array([
        [0, 0],
        [0, 1],
        [1, 0], 
        [1, 1]
    ])
Y = np.array([0, 1, 1, 0])
print(nn.predict([0, 0]))
print(nn.predict([0, 1]))
print(nn.predict([1, 0]))
print(nn.predict([1, 1]))

[ 0.32141961]
[ 0.59761275]
[ 0.78147525]
[ 0.87345311]


In [85]:
nn.fit(X, Y)

In [86]:
print(nn.predict([0, 0]))
print(nn.predict([0, 1]))
print(nn.predict([1, 0]))
print(nn.predict([1, 1]))

[ 0.0170538]
[ 0.97517594]
[ 0.97511299]
[ 0.03402422]
